In [2]:
import json
import numpy as np
import gezi
from gezi import tqdm

# did corpus

In [5]:
files = [
  '../input/train/behaviors.tsv',
  '../input/dev/behaviors.tsv',
]

uids = set()
with open('../input/did_corpus.txt', 'w') as out:
  for file in files:
    total = len(open(file).readlines())
    for line in tqdm(open(file), total=total):
      l = line.strip().split('\t')
      uid, history = l[1], l[-2]
      if uid in uids:
        continue
      uids.add(uid)
      print(history, file=out)

100%|██████████| 376471/376471 [00:01<00:00, 295551.94it/s]


# did corpus with test 

In [5]:
# 效果差不多 还是采用上面那个吧 或者为了安全 只用训练数据也可以 id类安全一些 
# 如果是引用title 等等 可以考虑都使用
files = [
  '../input/train/behaviors.tsv',
  '../input/dev/behaviors.tsv',
  '../input/test/behaviors.tsv',
]

uids = set()
with open('../input/did_corpus2.txt', 'w') as out:
  for file in files:
    total = len(open(file).readlines())
    for line in tqdm(open(file), total=total):
      l = line.strip().split('\t')
      uid, history = l[1], l[-2]
      if uid in uids:
        continue
      uids.add(uid)
      print(history, file=out)

100%|██████████| 2370727/2370727 [00:07<00:00, 312394.01it/s]


In [4]:
len(uids)

2232748

# word corpus

In [4]:
from transformers import AutoTokenizer

files = [
          '../input/train/news.tsv',
          '../input/dev/news.tsv',
          '../input/test/news.tsv'
        ]

model_name = 'bert-base-cased'
model = f'/home/gezi/data/lm/{model_name}'
tokenizer = AutoTokenizer.from_pretrained(model)

vocab = gezi.Vocab(f'{model}/vocab.txt', fixed=True)

dids = set()
with open('../input/corpus.txt', 'w') as out:
  for file_ in files:
    total = len(open(file_).readlines())
    for line in tqdm(open(file_), total=total):
      l = line.strip().split('\t')
      did, title, abstract = l[0], l[3], l[4]
      if did in dids:
        continue
      dids.add(did)

      if abstract:
        text = title + ' ' + abstract
      else:
        text = title
      tokens = tokenizer.encode(text)
      tokens =  tokens[1:-1]
      print(' '.join(map(vocab.key, tokens)), file=out)

100%|██████████| 120961/120961 [00:32<00:00, 3682.92it/s]


# click title corpus for bert

In [1]:
news_names = ['did', 'cat', 'sub_cat', 'title', 'abstract', 'url', 'title_entities', 'abstract_entities']

In [2]:
# 每个点击序列作为一个文档 一个文档 两个文档表示  一个title文档 一个 entity word 文档

In [3]:
# 首先生成文档信息 m[did] (title)\t(entities)

In [12]:
files = [
          '../input/news/news.tsv',
        ]

# with open('../input/bert_click_corpus.txt', 'w') as out:

titles_dict = {}
entities_dict = {}
for file_ in files:
  total = len(open(file_).readlines())
  for line in tqdm(open(file_), total=total):
    l = line.strip().split('\t')
    did, title, title_entities, abstract_entities = l[0], l[3], l[-2], l[-1]
    title_entities = json.loads(title_entities)
    abstract_entities = json.loads(abstract_entities)
    titles_dict[did] = title
    entities = []
    en_set = set()
    for m in title_entities:
      entities += [m['Label']]
      en_set.add(entities[-1].lower())
    for m in abstract_entities:
      if m['Label'].lower() not in en_set:
        entities += [m['Label']]
    entities_dict[did] = entities

100%|██████████| 130379/130379 [00:03<00:00, 37114.97it/s]


In [13]:
titles_dict

{'N88753': 'The Brands Queen Elizabeth, Prince Charles, and Prince Philip Swear By',
 'N45436': 'Walmart Slashes Prices on Last-Generation iPads',
 'N23144': '50 Worst Habits For Belly Fat',
 'N86255': "Dispose of unwanted prescription drugs during the DEA's Take Back Day",
 'N93187': "The Cost of Trump's Aid Freeze in the Trenches of Ukraine's War",
 'N75236': "I Was An NBA Wife. Here's How It Affected My Mental Health.",
 'N99744': 'How to Get Rid of Skin Tags, According to a Dermatologist',
 'N5771': 'Check Houston traffic map for current road conditions',
 'N124534': 'Should NFL be able to fine players for criticizing officiating?',
 'N51947': 'How to record your screen on Windows, macOS, iOS or Android',
 'N59220': "It's been Orlando's hottest October ever so far, but cooler temperatures on the way",
 'N17957': 'This Guy Altered His Diet and Training to Drop 65 Pounds and Pack on Muscle',
 'N40259': 'Chile: Three die in supermarket fire amid protests',
 'N42222': 'Best PS5 games: 

In [14]:
entities_dict

{'N88753': ['Prince Philip, Duke of Edinburgh',
  'Charles, Prince of Wales',
  'Elizabeth II'],
 'N45436': ['IPad', 'Walmart', 'Apple Inc.'],
 'N23144': ['Adipose tissue'],
 'N86255': ['Drug Enforcement Administration'],
 'N93187': ['Ukraine'],
 'N75236': ['National Basketball Association'],
 'N99744': ['Skin tag', 'Dermatology', "Reader's Digest"],
 'N5771': ['Houston'],
 'N124534': ['National Football League'],
 'N51947': ['Microsoft Windows', 'Android (operating system)'],
 'N59220': ['Orlando, Florida'],
 'N17957': [],
 'N40259': ['Chile', 'Santiago'],
 'N42222': ['PlayStation'],
 'N46520': ['Christmas'],
 'N40599': [],
 'N13152': ['WXII-TV', 'KPNX'],
 'N22273': [],
 'N107267': ['Philadelphia Eagles', 'DeSean Jackson', 'Chicago Bears'],
 'N30547': ['Ram Pickup', 'Cummins'],
 'N42639': [],
 'N54460': ['Weight gain'],
 'N117551': [],
 'N79856': ['Instagram'],
 'N72751': ['Michigan'],
 'N81543': ['Catherine, Duchess of Cambridge'],
 'N5149': [],
 'N10616': ['Newark Liberty Internatio

In [11]:
entities_dict['N21320']

['Jason Simmons']

In [29]:
files = [
  '../input/train/behaviors.tsv',
  '../input/dev/behaviors.tsv',
  '../input/test/behaviors.tsv',
]

histories = []
uids = set()
for file in files:
  total = len(open(file).readlines())
  for line in tqdm(open(file), total=total):
    l = line.strip().split('\t')
    id, uid, history = l[0], l[1], l[-2]
    # 避免test集合的历史穿越
    if uid in uids:
      continue
    uids.add(uid)
    histories.append(history)
    
np.random.shuffle(histories)
      
with open('../input/click_title_corpus.txt', 'w') as out:
  for history in tqdm(histories):
    for did in history.split():
      print(titles_dict[did], file=out)
    print(file=out)
    for did in history.split():
      if entities_dict[did]:
        print(' '.join(entities_dict[did]), file=out)
    print(file=out)

100%|██████████| 876956/876956 [01:07<00:00, 12931.64it/s]


In [3]:
from transformers import AutoTokenizer

model_name = 'bert-base-cased'
model = f'/home/gezi/data/lm/{model_name}'
tokenizer = AutoTokenizer.from_pretrained(model)

vocab = gezi.Vocab(f'{model}/vocab.txt', fixed=True)

file_ = '../input/click_title_corpus.txt'
with open('../input/click_title_corpus2.txt', 'w') as out:
  total = len(open(file_).readlines())
  for line in tqdm(open(file_), total=total):
    tokens = tokenizer.encode(line)
    tokens =  tokens[1:-1]
    if tokens:
      print(' '.join(map(vocab.key, tokens)), file=out)

 10%|█         | 2916722/29011640 [16:17<2:25:49, 2982.52it/s]


KeyboardInterrupt: 